# The working of model trained via optuna (script tuning_sieci_cdpr_low_conv.py)

In [35]:
import pandas as pd
import yfinance as yf
from datetime import date,timedelta
import warnings
warnings.filterwarnings("ignore")
start_date = "2022-01-01"
end_date = date.today().strftime("%Y-%m-%d")
ticker = "CDR.WA"

data = yf.download(ticker,start_date,end_date)
data = data[['High','Low','Open','Close']]
data.head()

[*********************100%***********************]  1 of 1 completed


,High,Low,Open,Close
Date,,,,
2022-01-03,201.899994,192.360001,192.880005,200.600006
2022-01-04,201.800003,197.699997,200.899994,198.460007
2022-01-05,204.000000,197.199997,198.000000,204.000000
2022-01-07,201.550003,195.880005,200.399994,198.639999
2022-01-10,199.660004,188.100006,198.600006,188.100006


In [36]:
from tuning_sieci_cdpr_low_conv import build_best_model
import optuna
study = optuna.load_study(study_name='cdpr_low_another_dim',storage='sqlite:///cdpt_conv.db')
model = build_best_model(study.best_params)

[*********************100%***********************]  1 of 1 completed

[[[192.36000061 192.88000488 200.6000061 ]
  [197.69999695 200.8999939  198.46000671]
  [197.19999695 198.         204.        ]
  ...
  [192.         193.         193.02000427]
  [189.69999695 192.96000671 191.        ]
  [188.69999695 191.5        191.        ]]

 [[197.69999695 200.8999939  198.46000671]
  [197.19999695 198.         204.        ]
  [195.88000488 200.3999939  198.63999939]
  ...
  [189.69999695 192.96000671 191.        ]
  [188.69999695 191.5        191.        ]
  [186.46000671 190.         194.        ]]

 [[197.19999695 198.         204.        ]
  [195.88000488 200.3999939  198.63999939]
  [188.1000061  198.6000061  188.1000061 ]
  ...
  [188.69999695 191.5        191.        ]
  [186.46000671 190.         194.        ]
  [187.         192.         187.74000549]]

 ...

 [[164.5        167.75       167.80000305]
  [168.         168.         170.5       ]
  [167.94999695 171.80000305 168.75      ]
  ...
  [159.6000061  162.1499939  160.75      ]
  [160.44999695 16

35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - loss: 1030342.1250 - root_mean_squared_error: 983.4697 - val_loss: 128593403904.0000 - val_root_mean_squared_error: 358599.2188 - learning_rate: 1.0000
Epoch 2/1000
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 32724.3418 - root_mean_squared_error: 178.4582 - val_loss: 60728288.0000 - val_root_mean_squared_error: 7792.8354 - learning_rate: 1.0000
Epoch 3/1000
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 22762.6777 - root_mean_squared_error: 145.5682 - val_loss: 1273909120.0000 - val_root_mean_squared_error: 35691.8633 - learning_rate: 1.0000
Epoch 4/1000
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30221.6484 - root_mean_squared_error: 163.0508 - val_loss: 372910.4062 - val_root_mean_squared_error: 610.6639 - learning_rate: 0.1000
Epoch 5/1000
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 787.7425 - root_mean_squared_error: 28.0238 - val_loss: 16419.4688 - val_root_mean_squared_error: 128.1385 - learning_rate: 0.1000
Epoch 6/1000
35/35 ━━

In [37]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=model.epoch,y=model.history['loss'], name = 'training'))
fig.add_trace(go.Scatter(x=model.epoch,y=model.history['val_loss'], name = 'validation'))
fig.update_layout(title = 'Training - mean squared error',
                  hovermode = 'x unified')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=model.epoch,y=model.history['root_mean_squared_error'], name = 'training'))
fig.add_trace(go.Scatter(x=model.epoch,y=model.history['val_root_mean_squared_error'], name = 'validation'))
fig.update_layout(title = 'Training - root mean squared error',
                  hovermode = 'x unified')
fig.show()

In [39]:
import numpy as np 
data.drop(columns = 'High',inplace = True)
pred_array = data.iloc[-study.best_params['lags']:]
result = model.model.predict(np.expand_dims(pred_array.values,axis = [0,3]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


Results saving

In [40]:
import os
file_path = 'Wyniki.csv'
if os.path.exists(file_path):
    a = pd.read_csv(file_path,index_col = 0)
    a.loc[date.today(),'Prediction'] = result[0]
    a.loc[date.today()-timedelta(1),'Real value'] = pred_array.iloc[-1,0]
    a.to_csv(file_path)
else:
    a = pd.DataFrame(columns=['Prediction','Real values'])
    a.loc[date.today(),'Prediction'] = result[0]
    a.to_csv(file_path)
    